In [1]:
from devito import Grid

shape = (10, )

grid = Grid(shape=shape)

In [2]:
from devito import TimeFunction, Function

u = TimeFunction(name='u', grid=grid, time_order=2, space_order=2)
v = TimeFunction(name='v', grid=grid, time_order=2, space_order=2)
m = Function(name='m', grid=grid)
damp = Function(name='damp', grid=grid)

In [11]:
fwpde = m * u.dt2 - u.laplace + damp * u.dt

# This discrete PDE can be solved in a time-marching way updating u(t+dt) from the previous time step
# Devito as a shortcut for u(t+dt) which is u.forward. We can then rewrite the PDE as 
# a time marching updating equation known as a stencil using customized SymPy functions
from devito import Eq, solve

fwstencil = Eq(u.forward, solve(fwpde, u.forward))

In [12]:
# Define the wave equation, but with a negated damping term
adjeqn = m * v.dt2 - v.laplace - damp * v.dt

# Use `solve` to rearrange the equation into a stencil expression
adjstencil = Eq(v.backward, solve(adjeqn, v.backward))

In [13]:
fwstencil

Eq(u(t + dt, x), -2.0*dt**3*u(t, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + dt**3*u(t, x - h_x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + dt**3*u(t, x + h_x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + 0.5*dt**2*h_x**2*damp(x)*u(t - dt, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + 2.0*dt*h_x**2*m(x)*u(t, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) - dt*h_x**2*m(x)*u(t - dt, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)))

In [14]:
adjstencil

Eq(v(t - dt, x), -2.0*dt**3*v(t, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + dt**3*v(t, x - h_x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + dt**3*v(t, x + h_x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + 0.5*dt**2*h_x**2*damp(x)*v(t + dt, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + 2.0*dt*h_x**2*m(x)*v(t, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) - dt*h_x**2*m(x)*v(t + dt, x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)))

In [28]:
from sympy import diff
from devito.symbolics import retrieve_functions

t = grid.stepping_dim
(x, ) = grid.dimensions
hx = x.spacing

fns = retrieve_functions(fwstencil)

for fn in fns:
    if not fn.is_TimeFunction:
        continue
    derivative = fwstencil.rhs.diff(fn).doit()
    print("Derivative w.r.t. ", fn, " is ", derivative.doit())
    print("****")

Derivative w.r.t.  u(t + dt, x)  is  0
****
Derivative w.r.t.  u(t, x - h_x)  is  dt**3/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x))
****
Derivative w.r.t.  u(t, x + h_x)  is  dt**3/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x))
****
Derivative w.r.t.  u(t, x)  is  -2.0*dt**3/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) + 2.0*dt*h_x**2*m(x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x))
****
Derivative w.r.t.  u(t - dt, x)  is  0.5*dt**2*h_x**2*damp(x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x)) - dt*h_x**2*m(x)/(0.5*dt**2*h_x**2*damp(x) + dt*h_x**2*m(x))
****


In [24]:
from devito import Operator
print(Operator(fwstencil, dle='noop', dse='noop'))

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict damp_vec, const float dt, const float h_x, struct dataobj *restrict m_vec, struct dataobj *restrict u_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m)
{
  float (*restrict damp) __attribute__ ((aligned (64))) = (float (*)) damp_vec->data;
  float (*restrict m) __attribute__ ((aligned (64))) = (float (*)) m_vec->data;
  float (*restrict u)[u_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]]) u_vec->data;
  for (int time = time_m, t0 = (time)%(3), t1 = (time + 1)%(3), t2 = (time + 2)%(3); time <= time_M; time += 1, t0 = (time)%(3), t1 = (time + 1)%(3), t2 = (time + 2)%(3))
  {
    struct timeval start_section0, end_se

In [23]:
print(Operator(adjstencil))

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict damp_vec, const float dt, const float h_x, struct dataobj *restrict m_vec, struct dataobj *restrict v_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m)
{
  float (*restrict damp) __attribute__ ((aligned (64))) = (float (*)) damp_vec->data;
  float (*restrict m) __attribute__ ((aligned (64))) = (float (*)) m_vec->data;
  float (*restrict v)[v_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[v_vec->size[1]]) v_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  for (int time = t